# PasDGalère

In [ ]:
df.to_csv('data_final.csv', index=False)

In [1]:
import pandas as pd
import numpy as np
from sklearn import model_selection, metrics, ensemble

In [2]:
import torch 
use_cuda = torch.cuda.is_available()
use_cuda

True

In [3]:
path = 'C:/Users/matteo/Desktop/accenture_data/'

## Reading Data

In [4]:
# products and cities
df_products = pd.read_csv(path + "product_attributes.csv", sep=",")

df_cities = pd.read_csv(path + "cities_data.csv", sep=",")
df_cities = df_cities[['city_from_name', 'city_to_name', 'distance']]
df_cities = df_cities.drop_duplicates()
df_cities2 = df_cities.copy()
df_cities2['city_from_name'] = df_cities['city_to_name']
df_cities2['city_to_name'] = df_cities['city_from_name']
df_cities = pd.concat([df_cities, df_cities2])

In [5]:
# read train data
df_orders = pd.read_csv(path + "orders.csv", sep=";")
df_orders['origin_port'] = df_orders['origin_port'].replace(['ATHENAS'], 'Athens')
df_orders['origin_port'] = df_orders['origin_port'].replace(['BCN'], 'Barcelona')
df_orders['logistic_hub'] = df_orders['logistic_hub'].fillna('Nohub')

df = df_orders.merge(df_products, how='left', left_on='product_id', right_on='product_id', sort=True)
df['weight'].fillna(df['weight'].median(), inplace = True)
df['material_handling'].fillna(6, inplace=True)

df = df.merge(df_cities, how='left', 
              left_on=('origin_port', 'logistic_hub'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df.rename(columns={'distance': 'distance_port_hub'}, inplace=True)
df = df.drop(['city_from_name', 'city_to_name'], axis=1)

df = df.merge(df_cities, how='left', 
              left_on=('logistic_hub', 'customer'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df.rename(columns={'distance': 'distance_hub_customer'}, inplace=True)
df = df.drop(['city_from_name', 'city_to_name'], axis=1)

df = df.merge(df_cities, how='left', 
              left_on=('origin_port', 'customer'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df = df.drop(['city_from_name', 'city_to_name'], axis=1)
df['distance_port_customer'] = np.where(df['logistic_hub'] == 'Nohub', df['distance'], 0)

df['distance_port_hub'].fillna(0, inplace=True)
df['distance_hub_customer'].fillna(0, inplace=True)
df['distance_port_customer'].fillna(0, inplace=True)

df['distance_port_hub'] = np.where(np.logical_and(df['distance_port_hub'] == 0, df['distance_port_customer'] == 0), 
                                   df['distance_port_hub'].median(), df['distance_port_hub'])

df['distance_hub_customer'] = np.where(np.logical_and(df['distance_hub_customer'] == 0, df['distance_port_customer'] == 0), 
                                       df['distance_hub_customer'].median(), df['distance_hub_customer'])

df = df.drop(['distance', 'order_id', 'product_id'], axis=1)
df['total_weight'] = df['weight'] * df['units']

#df = df.convert_dtypes()

In [6]:
# read train data
df_test = pd.read_csv(path + "test.csv", sep=";")
df_test['origin_port'] = df_test['origin_port'].replace(['ATHENAS'], 'Athens')
df_test['origin_port'] = df_test['origin_port'].replace(['BCN'], 'Barcelona')
df_test['logistic_hub'] = df_test['logistic_hub'].fillna('Nohub')

df_test = df_test.merge(df_products, how='left', left_on='product_id', right_on='product_id', sort=True)
df_test['weight'].fillna(df_test['weight'].median(), inplace = True)
df_test['material_handling'].fillna(6, inplace=True)

df_test = df_test.merge(df_cities, how='left', 
              left_on=('origin_port', 'logistic_hub'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df_test.rename(columns={'distance': 'distance_port_hub'}, inplace=True)
df_test = df_test.drop(['city_from_name', 'city_to_name'], axis=1)

df_test = df_test.merge(df_cities, how='left', 
              left_on=('logistic_hub', 'customer'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df_test.rename(columns={'distance': 'distance_hub_customer'}, inplace=True)
df_test = df_test.drop(['city_from_name', 'city_to_name'], axis=1)

df_test = df_test.merge(df_cities, how='left', 
              left_on=('origin_port', 'customer'), 
              right_on=('city_from_name', 'city_to_name'), sort=True)
df_test = df_test.drop(['city_from_name', 'city_to_name'], axis=1)
df_test['distance_port_customer'] = np.where(df_test['logistic_hub'] == 'Nohub', df_test['distance'], 0)

df_test['distance_port_hub'].fillna(0, inplace=True)
df_test['distance_hub_customer'].fillna(0, inplace=True)
df_test['distance_port_customer'].fillna(0, inplace=True)

df_test['distance_port_hub'] = np.where(np.logical_and(df_test['distance_port_hub'] == 0, df_test['distance_port_customer'] == 0), 
                                   df_test['distance_port_hub'].median(), df_test['distance_port_hub'])

df_test['distance_hub_customer'] = np.where(np.logical_and(df_test['distance_hub_customer'] == 0, df_test['distance_port_customer'] == 0), 
                                       df_test['distance_hub_customer'].median(), df_test['distance_hub_customer'])

df_test = df_test.drop(['distance', 'product_id'], axis=1)
df_test['total_weight'] = df_test['weight'] * df_test['units']

df_test = df_test.convert_dtypes()
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 114276 entries, 0 to 114275
Data columns (total 13 columns):
 #   Column                  Non-Null Count   Dtype  
---  ------                  --------------   -----  
 0   origin_port             114276 non-null  object 
 1   3pl                     114276 non-null  object 
 2   customs_procedures      114276 non-null  object 
 3   logistic_hub            114276 non-null  object 
 4   customer                114276 non-null  object 
 5   units                   114276 non-null  int64  
 6   late_order              114276 non-null  bool   
 7   weight                  114276 non-null  float64
 8   material_handling       114276 non-null  float64
 9   distance_port_hub       114276 non-null  float64
 10  distance_hub_customer   114276 non-null  float64
 11  distance_port_customer  114276 non-null  float64
 12  total_weight            114276 non-null  float64
dtypes: bool(1), float64(6), int64(1), object(5)
memory usage: 11.4+ MB


## Learning

In [7]:
# to convert categorical var
columns_to_dummies = ['origin_port','logistic_hub', 'customer', '3pl', 'customs_procedures']
df = pd.get_dummies(df, columns=columns_to_dummies)
df_test = pd.get_dummies(df_test, columns=columns_to_dummies)

In [8]:
# split data
X_train, X_validation, y_train, y_validation = model_selection.train_test_split(df.drop('late_order', axis=1), 
                                                                df['late_order'], test_size=0.20, random_state=79)
X_test = df_test

In [9]:
X_train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 91420 entries, 107396 to 38077
Data columns (total 55 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   units                    91420 non-null  int64  
 1   weight                   91420 non-null  float64
 2   material_handling        91420 non-null  float64
 3   distance_port_hub        91420 non-null  float64
 4   distance_hub_customer    91420 non-null  float64
 5   distance_port_customer   91420 non-null  float64
 6   total_weight             91420 non-null  float64
 7   origin_port_Athens       91420 non-null  uint8  
 8   origin_port_Barcelona    91420 non-null  uint8  
 9   origin_port_Rotterdam    91420 non-null  uint8  
 10  logistic_hub_Bratislava  91420 non-null  uint8  
 11  logistic_hub_Dusseldorf  91420 non-null  uint8  
 12  logistic_hub_Hamburg     91420 non-null  uint8  
 13  logistic_hub_Liege       91420 non-null  uint8  
 14  logistic_hub_Lill

In [10]:
# training model
classifier = ensemble.RandomForestClassifier(n_estimators=50, max_features="auto", random_state=79)
#classifier.fit(X_train, y_train)

In [11]:
#pred = classifier.predict(X_validation)
#metrics.accuracy_score(y_validation, pred)

### Méta-paramètres tuning

In [12]:
random_grid = {#'n_estimators' : [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)], 
               'max_features' : ['auto', 'sqrt'], 
               'max_depth' : [int(x) for x in np.linspace(30, 90, num = 10)], 
               'min_samples_split' : [10, 15, 20, 25, 30], 
               'min_samples_leaf' : [2, 3, 4, 5, 6], 
               'bootstrap' : [True, False]}

classifier_random = model_selection.RandomizedSearchCV(estimator = classifier, param_distributions=random_grid, n_iter=60, 
                                        cv=3, verbose=10, random_state=79, n_jobs=-1) 
classifier_random.fit(X_train, y_train)
best_params = classifier_random.best_params_
best_params

Fitting 3 folds for each of 60 candidates, totalling 180 fits


{'min_samples_split': 15,
 'min_samples_leaf': 4,
 'max_features': 'sqrt',
 'max_depth': 63,
 'bootstrap': True}

In [13]:
best_forest = ensemble.RandomForestClassifier(**best_params)
best_forest.fit(X_train, y_train)
pred = best_forest.predict(X_validation)
metrics.accuracy_score(y_validation, pred)

0.8138781939096955

In [17]:
'''
param_grid = { 
    'max_features': ['sqrt'], 
    'max_depth': [int(x) for x in np.linspace(53, 73, num = 5)],
    'min_samples_split' : [int(x) for x in np.linspace(10, 20, num = 5)],
    'min_samples_leaf' : [int(x) for x in np.linspace(3, 5, num = 3)], 
    'bootstrap': [True], 
}
'''
param_grid = { 
    'max_features': ['sqrt'], 
    'max_depth': [58],
    'min_samples_split' : [int(x) for x in np.linspace(10, 30, num = 15)],
    'min_samples_leaf' : [4], 
    'bootstrap': [True], 
}
rf = ensemble.RandomForestRegressor()
grid_search = model_selection.GridSearchCV(estimator = rf, param_grid = param_grid, 
                          cv=3, verbose=10, n_jobs=-1)

grid_search.fit(X_train, y_train)
best_grid = grid_search.best_estimator_
best_grid

Fitting 3 folds for each of 15 candidates, totalling 45 fits


RandomForestRegressor(max_depth=58, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=21)

## Save Solution
> **Note:** Submited probabilities must be for the **True** cases.

In [18]:
classifier = ensemble.RandomForestClassifier(n_estimators=1000, max_depth=58, max_features='sqrt', min_samples_leaf=4,
                      min_samples_split=21, bootstrap=True, random_state=79)
classifier.fit(X_train, y_train)
pred = classifier.predict(X_validation)
metrics.accuracy_score(y_validation, pred)

0.8134406720336017

In [19]:
pred_proba = classifier.predict_proba(X_test.drop('order_id', axis=1))
submission = pd.DataFrame({"order_id": X_test.order_id, "late_order": pred_proba[:,1]})
submission.to_csv("submission_kaggle.csv", index=False)

In [ ]:
n = [50, 100, 200, 500, 1000, 1500, 2000]
for i in range(7):
    classifier = ensemble.RandomForestClassifier(n_estimators=n[i], max_depth=35, max_features='sqrt', min_samples_leaf=2,
                      min_samples_split=42, bootstrap=True, random_state=79)
    classifier.fit(X_train, y_train)
    pred_proba = classifier.predict_proba(X_test.drop('order_id', axis=1))
    submission = pd.DataFrame({"order_id": X_test.order_id, "late_order": pred_proba[:,1]})
    submission.to_csv("submission_kaggle.csv" + str(i), index=False)